# review_crawling

## 필수 코드

In [ ]:
import requests
from tqdm import tqdm
import time
import json
import pandas as pd
from bs4 import BeautifulSoup
import re
import numpy as np
import time

## 크롤링 할 제품 종류 선정

### 1차

In [ ]:
# 화장품
# 트리트먼트 1000개
# 멀티밤 1000개
# 바디로션 1000개
# 핸드크림 1000개
# 틴트 1000개
# 스킨토너 1000개
# 에센스 1000개
# 앰플 1000개
# 썬크림 1000개
# 쿠션 파운데이션 1000개

# 전자기기
# 무선 마우스 2000개
# 유선 헤드셋 500개
# 무선 헤드셋 500개
# 두피 마사지기 1000개
# 손 마사지기 1000개
# 눈 마사지기 1000개
# 허리 마사지기 1000개
# 삼성 노트북 1000개
# 청소기 1000개
# 전동드릴 1000개

### 2차

In [ ]:
# 화장품
# BB크림 1000개
# CC크림 1000개
# 향수 1000개
# 네일케어 1000개
# 매니큐어 1000개
# 립스틱 1000개
# 아이팬슬 1000개
# 아이섀도 1000개
# 화장솜 1000개
# 쉐이빙폼 1000개

# 전자기기
# 무선물걸레청소기 1000개
# 유선물걸레청소기 1000개
# 유아용 전동칫솔 1000개
# 김치냉장고 1000개
# 클랜징마사지기 1000개
# 드라이기 1000개
# 유선 마우스 1000개
# 컴퓨터 1000개
# lg그램 1000개
# 태블릿 1000개

# 전자기기 5개
# 모니터 2000개
# 안마의자 2000개
# 유선 키보드 2000개
# 무선 키보드 2000개
# 모니터 암 2000개

## 크롤링 코드

### 각 제품별 번호

각 제품별 번호를 보는 방법은 특정 제품을 선택한 후(광고제외) url에서  
- https://search.shopping.naver.com/catalog/22420753111?&NaPm=ct%3Dlac7rsxc%7Cci%3D4f0c98debff98c49cba8dffa32a4e99d13b9fd76%7Ctr%3Dslcc%7Csn%3D95694%7Chk%3Dca1e4857423940377794d9a992ea088ffb1883f8   
- catalog 앞에 있는 번호를 확인 및 추가 하면 된다.

#### 1차

In [ ]:
# 각 제품별 번호
# 화장품
# 제품별 번호 트리트먼트, 멀티밤,    바디로션,  핸드크림,     틴트,       스킨토너,   에센스,       앰플 ,       썬크림,    쿠션 파운데이션
h_num = [21108679094, 22566448318, 15334437818, 6799239096, 25774739522, 20882947864, 24247070523, 24974942522, 14059639036, 26539564522]

# 전자기기
# 제품별 번호   무선 마우스,          유선   , 무선 헤드셋,  두피 마사지, 손마사지기, 눈마사지기, 허리마사지기, 삼성노트북,    청소기,    전동드릴
j_num = [15369599180, 32713692618, 1000041315, 32484298620, 24635272523, 20274260493, 32754338270, 15075924067, 32655347621, 30571433619, 26237963522]

In [ ]:
# 한번에 크롤링을 위한 모으기
1cha_num = h_num + j_num

#### 2차

In [ ]:
# 각 제품별 번호
# 화장품2 BB크림,      CC크림,      향수,       네일케어,   매니큐어,     립스틱,     아이팬슬,   아이섀도,     화장솜,     쉐이빙폼
h2_num =[26731766527, 25091557525, 8442841073, 30485507618, 20142129863, 5391724637, 19315199461, 27119781522, 22050101716, 22055523351]

# 전자기기2 무선물걸레청소기, 유선, 유아용전동칫솔, 김치냉장고,클랜징마사지기, 드라이기, 유선마우스, 컴퓨터,      lg그램,      태블릿
j2_num = [27808789522, 23310070490, 25425967522, 29219221618, 31042500618, 14480913735, 22742122709, 25052157522, 31437074618, 22280177621]

# 전자기기 모니터       안마의자,   유선 키보드, 무선 키보드, 모니터 암
j3_num = [13643814149, 24895107748, 11291155550, 20899895174, 22023989449]

In [ ]:
# 한번에 크롤링을 위한 모으기. 단, j3_num은 크롤링 갯수가 다르기 때문에 단독으로 가져감
2cha_num = h2_num + j2_num

### 데이터 크롤링 및 지정한 변수에 저장

In [ ]:
# 크롤링을 할 경우 들어갈 list 변수 지정
review1 = []
review2 = []
review3 = []

In [ ]:
# j3_num 자리에 1cha_num, 2cha_num을 적어주면 됨
for je in j3_num:
    # 총 리뷰 1000개를 가져오기 위해 range는 50으로(1당 리뷰는 20개). 단, j3_num인 경우 range는 100으로 지정
    for i in tqdm(range(50)):
        pg = i + 1

        # 리뷰가 적은 특정 제품을 위한 if문
        if (je == 1000041315 or je == 32484298620) and pg == 26:
            break
        
        # crawling를 위한 url
        url = "https://search.shopping.naver.com/api/review?"

        # json 형식으로 크롤링을 하기 위한 headers
        headers = {
            'referer': 'https://search.shopping.naver.com/catalog/{}?&NaPm=ct%3Dl93w1nyw%7Cci%3D4fb0cfd804a314df75fe80f0bc0678eada7e869e%7Ctr%3Dslcc%7Csn%3D95694%7Chk%3Debd5dc71f5929647e11ad13a5c4b9ede273cff44'.format(je),
            "User-Agent" : "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.0.0 Safari/537.36"}

        # json 형식으로 크롤링을 하기 위한 params
        params = {
                "nvMid" : "{}".format(je),
                #"topicCode" : "design", # 네이버 리뷰 중에서 디자인만 가져올 경우 "topicCode"앞에 #제거
                "reviewType":"ALL",
                "sort":"QUALITY",
                "isNeedAggregation":"N",
                "N&isApplyFilter":"N",
                "page":"{}".format(pg),
                "pageSize":"20"
            }

        
        resp = requests.get(url, params=params, headers=headers)

        # 지속적인 크롤링을 위한 2중 try_except
        try:
            time.sleep(0.5)
            dic = json.loads(resp.content)
        except:
            try:
                time.sleep(2)
                dic = json.loads(resp.content)
            except:
                time.sleep(5)
                dic = json.loads(resp.content)

        tags = dic['reviews']

        # 가져온 문장 or 문단을 특정 변수에 모으기
        for i in tags:
            review2.append(i['content']) # 1cha_num은 review1, 2cha_num은 review2, j3_num은 review3


100%|██████████| 100/100 [02:18<00:00,  1.38s/it]


In [ ]:
# review 갯수 확인
print(len(review1))
print(len(review2))
print(len(review3))

10000
10000
10000


In [ ]:
# 모든 review를 하나의 변수에 저장
reviews = review + review1 + review2
print(len(reviews))

30000


## 크롤링 데이터 -> 데이터 프레임화

In [ ]:
# 전처리를 위한 데이터 프레임화
df = pd.DataFrame(reviews)
# print(df)

In [ ]:
# 전처리 전 어떤 단어 또는 특수문자를 지울지 확인
print(df[0][19])
# <> 또는 <em></em>, </br> 또는 <br> 은 속성태그를 나타내므로 이모티콘과는 별개다, 전처리 할 때 지우기 위한 확인

제가 지성피부인데요! 인셀덤 알고 나서부터 유수분 밸런스 맞춰가고잇어요 속당김은 덜해졌구요 <em>유분도 점차 줄고</em> 잇습니다! 믿고 3개월째사용 중입니당! 비비 커버력이 우수하진않지만 <em>내피부마냥 표현하기 좋구요</em>! <em>일단 자연스럽고</em> 썬젤이랑 같이 쓰시면 완전 최곱니자 물광도 표현되구요!


## 전처리

In [ ]:
import re

df_concat = []
for i in tqdm(range(len(df[0]))):
  df[0][i] = df[0][i].replace('<em>','') # 문장에서 <em>제거
  df[0][i] = df[0][i].replace('</em>','') # 문장에서 </em>제거
  df[0][i] = df[0][i].replace('<br>','') # 문장에서 <br>제거
  df[0][i] = re.sub("[^가-힣ㄱ-ㅎㅏ-ㅢ .]",'',df[0][i]) # 문장에서 한글, 마침표(.)를 제외한 나머지 제거
  df[0][i] = df[0][i].split('.') # 문장을 마침표 기준으로 나누기
  df_concat += df[0][i] # 나눠준 문장들을 한 곳에 모아두기 위한 작업

100%|██████████| 30000/30000 [00:37<00:00, 794.21it/s]


In [ ]:
df_concat

['지읜의 소개로 쓰게되었습니다피부과도 성형외과도 안다니는 지인의 피부가 나이에비해너무어리고 예뻐서 알아보니 인셀덤을 몇년째쓰고있다고부랴부랴 전 첨부터 두세트를ㅈ구입해서신랑이랑 둘이같이쓰기시작했어요첨에 부스터랑크림이랑 일주일에 한통씩 쓸요량으로사용시재이틀째엔 레이저시술한듯 양볼과 이마가 화상을입은듯 쓰라리고 따갑고 그열감이 어마무시했습니다즉각 지인에게 조언을구하니 사용중지하고 젤만쓰면서 진정시키다 다시사용해보란 처방을받고그리 사용하니 다시 정상으로 돌아와서전 모공이 큰 피부인데 모공도 많이 좁혀지고 또 제가 필러시술을 받은지 이삼년되었는데 그 필러부분이 더 빵빵하게  차오른듯합니다대박만족중이고 꾸준히 쓰고있어요',
 '신랑과 함께쓰는데 신랑보단 제가 효과가 더좋네요케바케라생각됩니다전 아주 거실테이블위에 셋팅해놓고 티브시청하면서 바른고있네요구입주저하시는분들주저하지마시고 사용하시면 후회없으실듯해요',
 '전피부결이 좋지못해서 아무리 좋다는 팩트나 비비들을 발라도 뜨거나 바르고나서 피부가더안좋게  보여지는게 싫어서 비비나 팩트를 바르지 않는사람으로써 이제품은 지인소개로 알게됐는데 너무좋아서 글을썼네요  발림성은 모공끼임전혀없고뭉치거나 뜨는것도없어서 좋았어요발리는것도 촉촉하면서얇고자연스럽게 발리고 바르고나면 얼굴에 광은나는데 마스크에 묻어나지도않아요전 땀이많이 나는데 얼굴에 땀이나면 팩트로인해서 하얗게 올라면서 닦고나면 얼룩덜룩 보기안좋았던경험있으실거에요 이제품은 그런게 전혀 없고 땀만팅겨내는느낌 저피부진짜 안좋은사람인데  요즘은 피부좋아졌단말듯구있네요 돈받고쓴글아닙니다제가 예민한사람으로써 결이 안좋다면 전추천합니다',
 '안녕하세여 지인의 소개로 상품을 써 보게 된 인 입니다',
 ' 우선 저는 지독한 악건성으로 화장품을 고를때와 선택할때 굉장히 신중합니다',
 ' 일부좋타는 이름있는 명품라인 화장품부터 정말 하나하나쓰면서 제가 맞는 상품을 찾아 늘 써왔습니다',
 ' 허나 나이가 하나둘 먹다보니 화장품도 바꿔야 되는 시가 한번씩는 찾아오더라구여',
 ' 기존에 잘

In [ ]:
len(df_concat)

106070

## 전처리 후 공백, nan값 제거(결측치 제거)

In [ ]:
# 데이터프레임화 한다음 결측치로써 제거(리스트상태에서 제거하려고 했으나 제대로 지워지지 않음.)
df_concat = pd.DataFrame(df_concat)

In [ ]:
# 공백을 직접 nan값으로 변환후 제거
df_concat.replace('', np.nan, inplace=True)
df_concat.replace(' ', np.nan, inplace=True)
df_concat.replace('  ', np.nan, inplace=True)
df_concat.replace('   ', np.nan, inplace=True)
df_concat.replace('    ', np.nan, inplace=True)
df_concat.replace('     ', np.nan, inplace=True)
df_concat.replace('        ', np.nan, inplace=True)
df_concat = df_concat.dropna(axis = 0)
df_concat = df_concat.reset_index(drop=True)

문장 맨앞의 공백 제거

In [ ]:
# 문장의 처음 시작이 공백이면 공백 제거
for i in tqdm(range(len(df_concat[0]))):
    if df_concat[0][i][0] == ' ':
         blank = ''
        for j in range(len(df_concat[0][i])):
            if df_concat[0][i][j] == ' ':
                continue
            blank += df_concat[0][i][j]
        df_concat[0][i] = blank

df_concat[0][0]

100%|██████████| 81338/81338 [00:27<00:00, 2934.36it/s]


'먼저 다른분들의 상품평을 여러껀 읽어보고 구매한거라 그런지 제품이 낯설지가 않더라구요게다가 큰 기대에도 불구하고 아주 만족스럽습니다'

In [ ]:
df_concat

,0
0,먼저 다른분들의 상품평을 여러껀 읽어보고 구매한거라 그런지 제품이 낯설지가 않더라구...
1,구매 후 한번 사용해보고 작성한 후기랍니다
2,전 수요일에 진공청소기로 전체 청소를 한 뒤에금욜 아침에 약 분정도의 시간을 이용해...
3,사용전에 미리 하루정도 충전을 한터라 그런지평아파트를 베란다까지 전부 청소하기에 부...
4,네모진형태라 각진부분 청소도 좋았구요물조절만 잘하면 완벽하겠다 싶었습니다
...,...
81333,노트북받침대는좋네요
81334,사용한지 약 개월정도 되는데모니터 무게도 잘 버텨주고아주 편하게 쓰고 잇어여
81335,내구성이나 가성비눈 좋으나 틸트시 삐걱거리는 장력 조절은 싼 제품의 느낌이 남
81336,배송도 빠르고 제품 품질도 만족합니다


In [ ]:
# 하나의 문장을 선택해 문장의 맨앞이 공백인지 아닌지 체크
df_concat[0][8133]

'둘째 살 충치생겨서 사용중이에요'

In [ ]:
len(df_concat)

81338

In [ ]:
# 공백 또는 nan값이 전처리가 제대로 적용됬는지 확인 코드
count = 0
for i in range(len(df_concat)):
    if df_concat[0][i] == ' ':
        count += 1
count

0

## json 만들기 전 가공

In [ ]:
j_data = []
# 정해진 형태로 json 으로 변환하기 위한 작업
for i in tqdm(range(len(df_concat[0]))):
    j_data.append({"id":"", "sentence_form":df_concat[0][i], "annotation":[]})

100%|██████████| 81338/81338 [00:00<00:00, 125240.01it/s]


## json 전환 후 저장

### 데이터 프레임을 제외한 형식의(예를들어 list) 데이터를 json으로변환

In [ ]:
# 저장할 경로와 파일이름을 file_path에 저장
file_path = '../review_data.json'

# 저장된 내용으로 json화
with open(file_path, 'w') as j:
    json.dump(j_data, j)

### 데이터프레임을 json과 xlsx 로 변환

In [ ]:
# 엑셀화
df_concat.to_excel('../review_data.xlsx')
# json화
df_concat.to_json('../review_data.json')

## 저장된 json 테스트

In [ ]:
# 저장된 json을 불러와서 확인
test = pd.read_json('../review_data.json')
test

,id,sentence_form,annotation
0,,디자인도 이쁘고 가격도 싼편이라서 구매했습니다,[]
1,,사이즈는 좀 작은 편이더라고요,[]
2,,제 손이 작은편이라 만족했습니다,[]
3,,다만 처음에 컴퓨터 연결하고 난 직후에 오작동이 있었습니다,[]
4,,대각선 방향으로 마우스를 움직일때 마우스 커서가 자기 멋대로 튀는 현상이 있었습니다,[]
...,...,...,...
30991,,배송 오자마자 벽에 못 박으려고 타공시공 했는데요생각보다 힘도 좋구 잘 뚤립니다가...,[]
30992,,하나 가지고싶데서 선물로 줬는데 아주만족하네욥 파워도 세고 충전도 아주 잘된다고합니다,[]
30993,,사은품도 넘넘 맘에들어요,[]
30994,,일단 대만족 아직 써볼일이 없어서 안쓰고 있는데 시운전만 해봤습니다,[]
